# The Scoring Script 📝

When deploying a model as an endpoint in Azure Machine Learning, you need to provide a scoring script that will be used to make predictions. This script should contain two required functions:

1. `init()`: This function loads the model into memory when the service starts.
2. `run(input_data)`: This function uses the model to predict a value based on the input data.

**_Note_**: The `init()` and `run(input_data)` functions are required for the scoring script to work. You can add additional functions to the script as needed.

### The `init()` Function 🏁

- The `init()` function is called when the service is initialized. 

- In this function, you should load the model into memory so that it can be used for scoring. The path to the model file is provided by the `Model` object that is passed to the `init()` function as an input when you deploy the model to an endpoint. 

- Extensive logging is recommended in this function to help find the correct path to the model file and diagnose any potential issues that may arise.

In [ ]:
import os
import tensorflow as tf

def init():
    # Define the model as a global variable to be used later in the predict function
    global model

    # Get the path where the model is saved, it is set in the environment variable AZUREML_MODEL_DIR by the deployment configuration
    base_path = os.getenv("AZUREML_MODEL_DIR")
    print(f"base_path: {base_path}")
    
    # show the files in the model_path directory
    print(f"list files in the model_path directory")
    # list files and dirs in the model_path directory
    list_files(base_path)
    
    # add the model file name to the base_path
    model_path = os.path.join(base_path, 'model.keras') # local
    # model_path = os.path.join(base_path, "INPUT_model", 'model.keras') # azure
    # print the model_path to check if it is correct
    print(f"model_path: {model_path}")
    
    # Load the model
    model = tf.keras.models.load_model(model_path)
    print("Model loaded successfully")


# Helper function to list files in a directory
def list_files(startpath):
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, "").count(os.sep)
        indent = " " * 4 * (level)
        print("{}{}/".format(indent, os.path.basename(root)))
        subindent = " " * 4 * (level + 1)
        for f in files:
            print("{}{}".format(subindent, f))

### The `run(input_data)` Function 🏃

- The `run(input_data)` function is called when the endpoint is hit with an HTTP request.
- The input data is passed to this function as a JSON payload with the following format:
 `{"data": "base64-encoded-image-string"}`.
- We need to:
    - Decode the input data from base64.
    - Preprocess the input data.
    - Use the model to predict the class of the input image.
    - Postprocess the prediction (if necessary)
    - Return the prediction as a JSON payload with the following format:
    `{"result": "predicted-class"}`.
 

**_Note_**: It is advised to test this with a local deployment before deploying the model as an endpoint in Azure Machine Learning. That way you can build and debug the scoring incrementally using the logs to understand the flow of data and predictions.

In [ ]:
import json
import base64
import io
from PIL import Image
import numpy as np

def run(raw_data):

    # Load the JSON data from the POST request, print the data to see the structure and content
    print(f"raw_data: {raw_data}")
    data = json.loads(raw_data)
    print(f"data: {data}")

    # Get the base64-encoded image data, print the data to see make sure it is correct
    base64_image = data["data"]
    print(f"base64_image: {base64_image}")

    # Decode the base64 string into bytes
    image_bytes = base64.b64decode(base64_image)
    print(f"image_bytes: {image_bytes}")

    # Open the bytes as an image
    image = Image.open(io.BytesIO(image_bytes))

    # Preprocess the image
    # Convert the image to grayscale
    image = image.convert("L")
    # Resize the image to 28x28 pixels, the size expected by the model
    image = image.resize((28, 28))
    # Convert the image to a numpy array and normalize pixel values to [0, 1]
    data = np.array(image) / 255.0
    # The model expects a 4D tensor of shape (batch_size, height, width, channels),
    # so we add an extra dimension to the start and end of the array
    data = np.expand_dims(data, axis=(0, -1))

    # Make prediction, print the prediction to see the structure and content
    prediction = model.predict(data)
    print(f"prediction: {prediction}")
    # Get the predicted label
    predicted_label = np.argmax(prediction, axis=1)[0]

    # Print the predicted label
    print(f"Predicted label: {predicted_label}")
    print(f"Output format: {json.dumps(predicted_label.tolist())}")

    return json.dumps(predicted_label.tolist())